### Solutions to Exercises

### Solution 1: Simple Exercise
Enhance the script to also log humidity data and visualize both temperature and humidity on the same plot.

In [ ]:
import Adafruit_DHT
import time
import smtplib
from email.mime.text import MIMEText
import matplotlib.pyplot as plt

# Sensor setup
DHT_SENSOR = Adafruit_DHT.DHT11
DHT_PIN = 4

# Email setup
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587
EMAIL_ADDRESS = 'your_email@gmail.com'
EMAIL_PASSWORD = 'your_password'

def send_email(temp):
    subject = 'Temperature Alert'
    body = f'Temperature has exceeded the threshold! Current temperature is {temp}°C.'
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = EMAIL_ADDRESS

    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        server.sendmail(EMAIL_ADDRESS, EMAIL_ADDRESS, msg.as_string())

# Function to read sensor data
def read_sensor():
    humidity, temperature = Adafruit_DHT.read(DHT_SENSOR, DHT_PIN)
    if humidity is not None and temperature is not None:
        return temperature, humidity
    else:
        return None, None

# Function to log data to a file
def log_data(temperature, humidity):
    with open("temperature_log.txt", "a") as file:
        file.write(f"Temp={temperature}C  Humidity={humidity}%\n")

# Main loop
while True:
    temp, hum = read_sensor()
    if temp is not None and hum is not None:
        log_data(temp, hum)
        if temp > 25:
            send_email(temp)
    time.sleep(60)  # Log data every 60 seconds

# Read data from the log file
def read_log():
    temperatures = []
    humidities = []
    with open("temperature_log.txt", "r") as file:
        for line in file:
            if line.startswith("Temp"):
                temp = float(line.split()[0].split('=')[1][:-1])
                hum = float(line.split()[1].split('=')[1][:-1])
                temperatures.append(temp)
                humidities.append(hum)
    return temperatures, humidities

# Plot data
temperatures, humidities = read_log()
plt.plot(temperatures, label='Temperature (C)')
plt.plot(humidities, label='Humidity (%)')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Temperature and Humidity Over Time')
plt.legend()
plt.show()